<a href="https://colab.research.google.com/github/rhiga2/AIHawaii/blob/main/workshops/intro_drl/DRLTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install stable-baselines3
!apt install swig
!apt install xvfb python3-opengl ffmpeg
!pip install gymnasium[box2d]
!pip install pyvirtualdisplay
!pip install huggingface-sb3

In [3]:
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
display = Display(visible=0, size=(400, 300))
display.start()

In [4]:
import glob
import io
import base64
from IPython.display import HTML

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else:
    print("Could not find video")

In [5]:
# Log into huggingface
from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login
notebook_login()
!git config --global credential.helper store

In [6]:
import gymnasium as gym
from stable_baselines3.common.env_util import make_vec_env

env_id = "LunarLander-v2"
def make_env():
  return gym.make(
    env_id,
    render_mode="rgb_array",
  )

env = make_env()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
from gym.wrappers import RecordVideo
record_env = RecordVideo(env, './video')
record_env.reset()

terminated = False
while not terminated:
  action = record_env.action_space.sample()
  obs, reward, terminated, info = record_env.step(action)

record_env.close()
show_video()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.is_vector_env to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.is_vector_env` for environment variables or `env.get_wrapper_attr('is_vector_env')` that will search the reminding wrappers.
  logger.warn(


In [9]:
from stable_baselines3 import PPO

arch = "ppo"
model_name = "{arch}-{env}".format(arch=arch, env=env_id)
repo_id = "rhiga/{model_name}".format(model_name=model_name)

In [6]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

net_arch = [128, 128]
env = make_env()

model = PPO("MlpPolicy", env, batch_size=64, learning_rate=1e-3,
            policy_kwargs=dict(
              net_arch=net_arch
            ))
callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=250,
                                                 verbose=1)
eval_callback = EvalCallback(env, callback_on_new_best=callback_on_best,
                             verbose=1)
model.learn(total_timesteps=int(1e6), log_interval=10,
            callback=eval_callback, progress_bar=True)
model.save(model_name)

Output()

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation 
environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and 
rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(

Eval num_timesteps=10000, episode_reward=-297.82 +/- 146.40

Episode length: 254.40 +/- 50.93

New best mean reward!

...

Episode length: 502.00 +/- 98.12

Eval num_timesteps=350000, episode_reward=250.03 +/- 14.50

Episode length: 380.80 +/- 20.45

New best mean reward!

Stopping training because the mean reward 250.03  is above the threshold 250

In [10]:
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor

eval_env = DummyVecEnv([lambda: Monitor(gym.make(env_id, render_mode="rgb_array"))])
package_to_hub(
    model=model,
    model_name=model_name,
    model_architecture=arch,
    env_id=env_id,
    eval_env=eval_env,
    repo_id="rhiga/dqn-LunarLander-v2",
    commit_message="Initial commit"
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmp9yykrhus/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmp9yykrhus/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmp9yykrhus/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmp9yykrhus/-step-0-to-step-1000.mp4
ℹ Pushing repo rhiga/dqn-LunarLander-v2 to the Hugging Face Hub


policy.optimizer.pth:   0%|          | 0.00/298k [00:00<?, ?B/s]

policy.pth:   0%|          | 0.00/148k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

ppo-LunarLander-v2.zip:   0%|          | 0.00/461k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/rhiga/dqn-LunarLander-v2/tree/main/


'https://huggingface.co/rhiga/dqn-LunarLander-v2/tree/main/'

In [10]:
artifact = load_from_hub(
    repo_id,
    "{}.zip".format(model_name)
)
model = PPO.load(artifact)

ppo-LunarLander-v2.zip:   0%|          | 0.00/148k [00:00<?, ?B/s]

In [11]:
env = make_env()
record_env = RecordVideo(env, './video')
obs, info = record_env.reset()
terminated = False
while not terminated:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, info = record_env.step(action)
record_env.close()
show_video()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/record_video.py:78: UserWarning: WARN: Overwriting existing videos at /content/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.is_vector_env to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.is_vector_env` for environment variables or `env.get_wrapper_attr('is_vector_env')` that will search the reminding wrappers.
  logger.warn(
